In [ ]:
import torch
import torch.autograd
import numpy as np
import matplotlib.pyplot as plt
import numml.sparse as sp
import numml.krylov as kry

In [ ]:
# Create our favorite poisson operator

N = 8
A = sp.eye(N)*2. - sp.eye(N, k=1) - sp.eye(N, k=-1)
print(A.to_dense())

In [ ]:
# Create our starting initial guess to A^{-1}, which we'll be lazy and initialize to A itself

Ainv = A.copy() + sp.eye(N, k=2) + sp.eye(N, k=-2)
Ainv_start = Ainv.copy()
Ainv.requires_grad = True
Ainv.to_dense()

In [ ]:
(A@Ainv).to_dense()

In [ ]:
# Confirm our matmat product even works at all

torch.allclose((A@Ainv).to_dense(), A.to_dense() @ Ainv.to_dense())

In [ ]:
# Optimize || A @ Ainv - I ||_F^2 + || Ainv @ A - I ||_F^2

optimizer = torch.optim.Adam([Ainv.data], lr=0.1)
lh = []
epochs = 300

for i in range(epochs):
    optimizer.zero_grad()
    
    loss_AAinv = (((A @ Ainv) - sp.eye(N)) ** 2.).sum()
    loss_AinvA = (((Ainv @ A) - sp.eye(N)) ** 2.).sum()
    loss = loss_AAinv + loss_AinvA
    loss.backward()
    
    optimizer.step()
    
    lh.append(loss.item())
    if i % 10 == 0 or i == epochs - 1:
        print(i, loss.item())

In [ ]:
plt.semilogy(lh)
plt.grid()
plt.title('Loss history')

In [ ]:
(A@Ainv).to_dense()

In [ ]:
Ainv.to_dense()